# **Hyperparameter Tuning With Keras**

In [6]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import os
import warnings

warnings.filterwarnings("ignore")

# set tensorflow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL '] = '2'

In [7]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Load the MNIST dataset
(x_all, y_all), _ = keras.datasets.mnist.load_data()

# Flatten and normalize the images
x_all = x_all.reshape((x_all.shape[0], -1)).astype("float32") / 255.0

# Split into train+val and test (80/20)
x_temp, x_test, y_temp, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42)

# split train+val into train and validation (75/25 of 80%)
x_train, x_val, y_train, y_val = train_test_split(x_temp, y_temp, test_size=0.25, random_state=42)

In [8]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


## Defining the model with hyperparameters  

In [ ]:
def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

automaticallly find the best combination of hyperparamter for best validation accuracy

## Configuring the Hyperparamter search

In [10]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,
    directory='my_dir',
    project_name='mnist_tuning'
)

# Display a summary of the search space
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


- build_model, : the model-building function
- objective='val_accuracy', : the metric to optimize (validation accuracy)
- max_trials=5, : the maximum number of different hyperparameter configuration to try
- executions_per_trial=2, : the number of times to run each configuration
- directory='my_dir', : direcotry to save the result
- project_name='mnist_tuning' : name of the project for organizing result

## Running the hyper parameter search

In [11]:
# Run the hyperparameter search
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

tuner.results_summary()

Trial 5 Complete [00h 00m 46s]
val_accuracy: 0.9787000119686127

Best val_accuracy So Far: 0.9808499813079834
Total elapsed time: 00h 03m 58s
Results summary
Results in my_dir\mnist_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
units: 512
learning_rate: 0.0011852858827093944
Score: 0.9808499813079834

Trial 4 summary
Hyperparameters:
units: 256
learning_rate: 0.000768513725396424
Score: 0.9787000119686127

Trial 2 summary
Hyperparameters:
units: 320
learning_rate: 0.0002566590573584278
Score: 0.976500004529953

Trial 1 summary
Hyperparameters:
units: 64
learning_rate: 0.003081373838058736
Score: 0.9722500145435333

Trial 0 summary
Hyperparameters:
units: 256
learning_rate: 0.0001350192582783121
Score: 0.9654499888420105


## Analyzing and Using the best hyperparameters

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f""" 
The optimal number of units in the first dense layer is {best_hps.get('units')}. 
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}. 
""") 

 The optimal number of units in the first dense layer is 512. 
The optimal learning rate for the optimizer is 0.0011852858827093944. 



In [17]:
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9360 - loss: 0.2156 - val_accuracy: 0.9548 - val_loss: 0.1454
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9722 - loss: 0.0894 - val_accuracy: 0.9703 - val_loss: 0.0967
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9826 - loss: 0.0562 - val_accuracy: 0.9743 - val_loss: 0.0859
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9870 - loss: 0.0396 - val_accuracy: 0.9745 - val_loss: 0.0896
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9912 - loss: 0.0281 - val_accuracy: 0.9786 - val_loss: 0.0842
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9927 - loss: 0.0220 - val_accuracy: 0.9784 - val_loss: 0.0909
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9940 - loss: 0.0176 - val_accuracy: 0.9758 - val_loss: 0.0978
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9944 - loss: 0.0159 - 

In [18]:
# Evaluate the model on the test set 
test_loss, test_acc = model.evaluate(x_val, y_val) 
print(f'Test accuracy: {test_acc}') 

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9795 - loss: 0.0879
Test accuracy: 0.9794999957084656
